In [ ]:
import pandas as pd
import numpy as np
import mne
import matplotlib as mpl
from matplotlib import pyplot as plt
mpl.use('QtAgg')
from datetime import datetime, timedelta
from pathlib import Path

fix_electrode_names(raw)

Rename old electrode names to new standard and keep only the 19 standard electrodes.

Arguments:
    raw (mne.io.Raw): Raw EEG data

Returns:
    mne.io.Raw: Processed EEG data with correct electrode names

In [10]:
def fix_electrode_names(raw):
    # Create a copy to avoid modifying the original
    raw_processed = raw.copy()

    # Define the standard 19 electrodes template
    standard_electrodes = {'FP1', 'FP2', 'F3', 'F4', 'C3', 'C4', 'P3', 'P4',
                           'O1', 'O2', 'F7', 'F8', 'T7', 'T8', 'P7', 'P8',
                           'FZ', 'CZ', 'PZ'}

    # Define the channel name mapping for old to new names
    channel_mapping = {
        'T3': 'T7',
        'T4': 'T8',
        'T5': 'P7',
        'T6': 'P8'
    }

    # Check and rename channels if necessary
    current_channels = raw_processed.ch_names
    channels_to_rename = {}

    for old_name, new_name in channel_mapping.items():
        if old_name in current_channels:
            channels_to_rename[old_name] = new_name
            print(f"Renaming channel {old_name} to {new_name}")

    if channels_to_rename:
        raw_processed.rename_channels(channels_to_rename)

    # Get current channels after renaming
    current_channels = raw_processed.ch_names

    # Find channels to drop (those not in standard_electrodes)
    channels_to_drop = [ch for ch in current_channels if ch not in standard_electrodes]

    if channels_to_drop:
        print(f"Dropping non-standard channels: {channels_to_drop}")
        raw_processed.drop_channels(channels_to_drop)

    # Verify final channel set
    final_channels = raw_processed.ch_names
    print(f"Final channels ({len(final_channels)}): {final_channels}")

    return raw_processed


preprocess_eeg(raw)

Preprocess EEG data with standard pipeline (bandpass and avg reference).

Args:
    raw (mne.io.Raw): Raw EEG data

Returns:
    mne.io.Raw: Preprocessed EEG data

In [11]:
def preprocess_eeg(raw):
    
    # Create a copy to avoid modifying the original
    raw_processed = raw.copy()

    # Fix electrode names and keep only standard ones
    raw_processed = fix_electrode_names(raw_processed)

    # Apply bandpass filter (0.5-40 Hz)
    raw_processed.filter(l_freq=0.5, h_freq=40)
    print("Applied bandpass filter (0.5-40 Hz)")

    # Set average reference
    raw_processed.set_eeg_reference(ref_channels='average')
    print("Applied average reference")

    return raw_processed

compute_power_spectrum(raw_processed)

Compute power spectrum and analyzed frequency bands for each channel.

Args:
    raw_processed (mne.io.Raw): Preprocessed EEG data

Returns:
    pd.DataFrame: Results containing power values for different frequency bands

In [12]:
def compute_power_spectrum(raw_processed):
    
    # Get channel names
    channels = raw_processed.ch_names

    # Initialize lists to store results
    results = []

    # Helper function to get power in specific frequency range
    def get_freq_power(psds, freqs, fmin, fmax):
        idx = np.logical_and(freqs >= fmin, freqs <= fmax)
        return np.mean(psds[:, idx])

    # Calculate power spectrum for each channel
    for channel in channels:
        # Get the spectrum object
        spectrum = raw_processed.compute_psd(
            method='welch',
            picks=channel,
            fmin=0.5,
            fmax=40,
            n_fft=int(raw_processed.info['sfreq'] * 4),
            n_overlap=int(raw_processed.info['sfreq'] * 2)
        )

        # Get the data from spectrum
        psds = spectrum.get_data()
        freqs = spectrum.freqs

        # Calculate powers for different frequency bands
        delta_power_0p5_2 = get_freq_power(psds, freqs, 0.5, 2)
        delta_power_1_4 = get_freq_power(psds, freqs, 1, 4)
        total_power = get_freq_power(psds, freqs, 0.5, 40)

        # Store results
        results.append({
            'channel': channel,
            'delta_power_0p5_2': delta_power_0p5_2,
            'delta_power_1_4': delta_power_1_4,
            'total_power': total_power
        })

    # Create DataFrame from results
    df_results = pd.DataFrame(results)
    return df_results


analyze_delta_power(raw)

Main function to analyze delta power in EEG data.

Args:
    raw (mne.io.Raw): Raw EEG data

Returns:
    pd.DataFrame: Results of power analysis

In [13]:
def analyze_delta_power(raw):
    
    # 1. Preprocess the data
    raw_processed = preprocess_eeg(raw)

    # 2. Compute power spectrum and analyze
    df_results = compute_power_spectrum(raw_processed)

    # 3. Save results
    output_path = 'D:/seizures_analysis/excel_files'
    df_results.to_csv(output_path, index=False)
    print(f"Analysis complete. Results saved to {output_path}")

    return df_results

seizure_num_to_raw_data(pat_num,seizure_num)

Finding paths to data.

Args:
    Patient id and one seizure number of his

Returns:
    Raw EEG data and path to the specific patient list of seizures

In [14]:
def seizure_num_to_raw_data(pat_index,seizure_index):
    pat_num = str(pat_index)

    data_path = Path("E:/Ben Gurion University Of Negev Dropbox/CPL lab members/epilepsy_data/Epilepsiea")
    seizures_list_path = data_path / "tables" / "seizure_tables" / f"{pat_num}_surf30_seizures"
    seizures_list_table = pd.read_excel(seizures_list_path)
    seizures_data_path = data_path / "tables" / f"pat_{pat_num}_surf30_file_list"
    seizure_rec_num = seizures_list_table.loc[seizures_list_table['seizure_num'] == seizure_index, 'file_seizure_ind'].iloc[0]

    seizures_data_table = pd.read_excel(seizures_data_path)
    seizure_recording_path = seizures_data_table.loc[(seizure_rec_num + 2), 'file_path']
    raw = mne.io.read_raw_nicolet(seizure_recording_path)
    
    return raw, seizures_list_path

copy_and_crop(raw,seizures_list_path,seizure_ind,sec_before = 60, sec_after = 60):

Finding paths to data.

Args:
    Raw data of recording with a seizure in it, list of seizures' info, seizure index 

Returns:
    Raw EEG data crop of the seizure's time +- sec variant (default 60)

In [ ]:
def copy_and_crop(raw, seizures_list_path, seizure_ind, sec_before = 60, sec_after = 60):
    # Create a copy to avoid modifying the original
    raw = raw.copy()
    seizures_data = pd.read_csv(seizures_list_path)
    
    # Get recording information in seconds
    recording_duration = raw.times[-1]  # or: len(raw.times) / raw.info['sfreq']
    recording_start = np.datetime64(raw.info['meas_date'])
    
    # Get the start & end time of desired seizure
    seizure_start = pd.to_datetime(seizures_data.loc[seizure_ind, 'onset'], format='%d/%m/%Y %H:%M').to_numpy()
    seizure_end = pd.to_datetime(seizures_data.loc[seizure_ind, 'offset'], format='%d/%m/%Y %H:%M').to_numpy()
    
    # Subtraction of start and end from recording start to get duration in sec
    seizure_start_from_tmin = (seizure_start - recording_start) / np.timedelta64(1,'s')
    seizure_end_from_tmin = (seizure_end - recording_start) / np.timedelta64(1,'s')
    
    # Calculate crop times in sec (5 minutes before and after (60s))
    crop_start = seizure_start_from_tmin - sec_before
    crop_end = seizure_end_from_tmin + sec_after
    print(crop_start)
    print(crop_end)
    
    # Make sure we're within recording borders
    if crop_start < 0:
        crop_start = 0
        print('Warning: crop_start adjusted to recording start (0)')
    
    if crop_end > recording_duration:
        crop_end = recording_duration
        print(f'Warning: crop_end adjusted to recording end ({recording_duration}s)')
    
    # Cropping the data
    raw_cropped = raw.copy().crop(tmin=crop_start,tmax=crop_end)
    # Maybe: raising assert for data exception

Asserts for prev def 
Assert that seizure timing makes sense

    assert seizure_end_from_tmin > seizure_start_from_tmin, "Seizure end time is before start time"
    assert seizure_start_from_tmin >= 0, "Seizure starts before recording"
    assert seizure_end_from_tmin <= recording_duration, "Seizure ends after recording"
    
Assert final crop times are valid

    assert 0 <= crop_start < crop_end <= recording_duration, "Invalid crop times after adjustment"

# main

In [15]:
if __name__ == "__main__":
    try:
        # Load your EEG data
        raw = mne.io.read_raw_nicolet('/Users/maya/Documents/backup_lab_project/data/100102_0075.data', ch_type='eeg', preload=True)

        # Run the analysis
        results = analyze_delta_power(raw)
        print("\nFinal Results:")
        print(results)

    except Exception as e:
        print(f"An error occurred: {e}")

Reading header...
Reading 0 ... 921599  =      0.000 ...  3599.996 secs...
Renaming channel T3 to T7
Renaming channel T4 to T8
Renaming channel T5 to P7
Renaming channel T6 to P8
Dropping non-standard channels: ['SP1', 'SP2', 'RS', 'T1', 'T2', 'EOG1', 'EOG2', 'EMG', 'ECG', 'PHO']
Final channels (19): ['FP1', 'FP2', 'F3', 'F4', 'C3', 'C4', 'P3', 'P4', 'O1', 'O2', 'F7', 'F8', 'T7', 'T8', 'P7', 'P8', 'FZ', 'CZ', 'PZ']
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 1691 samples (6.605 s)

Applied bandpass 